In [1]:
import os

import random
import cv2
import numpy as np
import pandas as pd

from skimage.external import tifffile

from tqdm.auto import tqdm

#### Radboud: Prostate glands are individually labelled. Valid values are:

0: background (non tissue) or unknown

1: stroma (connective tissue, non-epithelium tissue)

2: healthy (benign) epithelium

3: cancerous epithelium (Gleason 3)

4: cancerous epithelium (Gleason 4)

5: cancerous epithelium (Gleason 5)


#### Karolinska: Regions are labelled. Valid values are:

1: background (non tissue) or unknown

2: benign tissue (stroma and epithelium combined)

3: cancerous tissue (stroma and epithelium combined)


In [2]:
colors = {
    'radboud': {0: '#ffffff',
                1: '#8080ff',
                2: '#80ff80',
                3: '#ff8080',                
                4: '#ff4040',                
                5: '#ff0000'},
    'karolinska': {0: '#ffffff',
                   1: '#808080',
                   2: '#ff0000'},
}

def hex2rgb(h):
    h = h.lstrip('#')
    return [int(h[i:i+2], 16) for i in (0, 2, 4)]

In [3]:
wsi_csv_path = "/mnt/HDDData/pdata/train.csv"
wsi_masks_path = "/mnt/HDDData/pdata/train_label_masks/"
wsi_rgb_masks_path = "/mnt/HDDData/pdata/train_label_rgb_masks/"

In [4]:
os.makedirs(wsi_rgb_masks_path, exist_ok=True)

In [5]:
wsi_df = pd.read_csv(wsi_csv_path)

In [6]:
fnames = os.listdir(wsi_masks_path)

In [7]:
fnames.sort()

In [8]:
random.shuffle(fnames)

In [ ]:
for fname in tqdm(fnames):
    image_id = fname.split('_')[0]
    img_row = wsi_df[wsi_df['image_id'] == image_id].iloc[0] 
    
    full_fname = os.path.join(wsi_masks_path, f"{image_id}_mask.tiff")
    n_img = tifffile.imread(full_fname)
    o_img = n_img[..., 0]
    # n_img = np.zeros(o_img.shape + (3,), dtype=np.uint8)
    
    for i in range(6):
        k = o_img == i
        if k.sum() > 0:
            n_img[k] = hex2rgb(colors[img_row.data_provider][i])
        
    for i, (k, v) in enumerate(colors[img_row.data_provider].items()):
        n_img[10:110, 10+i*110:(i+1)*110] = hex2rgb(v)

    cv2.putText(n_img, f"{img_row.data_provider}, grade: {img_row.isup_grade}, g_score: {img_row.gleason_score}",
                (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 5, 0, thickness=8);

    cv2.imwrite(os.path.join(wsi_rgb_masks_path, f"{image_id}_mask.tiff"), 
                cv2.cvtColor(n_img, cv2.COLOR_RGB2BGR))
    
    del o_img, n_img